In [ ]:
import os

In [ ]:
import pandas as pd
import numpy as np
import string
import json
table = str.maketrans('', '', string.punctuation)
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
Stemmer=SnowballStemmer("english", ignore_stopwords=True)
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import re

In [ ]:
CUR_DIR = os.path.abspath("")
DATA_DIR = os.path.join(CUR_DIR, "../../data")

In [ ]:
def word_count(str):
    counts = dict()
    str=remove_stopwords(str)
    words = word_tokenize(str)
    #removing stop words
    for word in words:
        #word = Stemmer.stem(word)
        word= wordnet_lemmatizer.lemmatize(word)
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
    return counts

In [ ]:
passages_path = os.path.join(DATA_DIR, "raw/WikiPassageQA/document_passages.json")
with open(passages_path) as f:
    data = json.load(f)

In [ ]:
query_path = os.path.join(DATA_DIR, "raw/extracted_query_data/train_exp.csv")
df = pd.read_csv(query_path)

In [ ]:
df

In [ ]:
docid=str(df.iloc[0][0])
relp = str(df.iloc[0][4])
question=df.iloc[0][3].translate(table).lower()
passage=data[docid][relp].translate(table).lower()
passage=re.sub(r'\n', ' ', passage)

In [ ]:
q = word_count(question)
p = word_count(passage)
setA = set(q)
setB = set(p)
print("The overlapping words between question and passage is:- ")
print(setA.intersection(setB))
print("Total number of overlapping words: "+str(len(setA.intersection(setB))))
print("Counts of the overlapping words in question and passage:-")
for item in setA.intersection(setB):
    print("Q count for the word '"+ item +"' is "+str(q[item]))
    print("P count for the word '"+ item +"' is "+str(p[item]))

In [ ]:
counts=[]
Qcounts=[]
Pcounts=[]
for i in range(len(df)):
    docid=str(df.iloc[i][0])
    relp = str(df.iloc[i][4])
    question=df.iloc[i][3].translate(table).lower()
    passage=data[docid][relp].translate(table).lower()
    passage=re.sub(r'\n', ' ', passage)
    q = word_count(question)
    p = word_count(passage)
    setA = set(q)
    setB = set(p)
    intersect = setA.intersection(setB)
    resQ = {key: q[key] for key in intersect} 
    resP = {key: p[key] for key in intersect}
    Qcounts.append(resQ)
    Pcounts.append(resP)
    counts.append(len(intersect))

In [ ]:
counts = np.array(counts)
counts.mean(),counts.std()

In [ ]:
list(Qcounts[0]),list(Pcounts[0])

In [ ]:
# Analyzing for the overlapping words how much they occur in the passages on average. 
means=[]
stds=[]
for i in range(len(Pcounts)):
    if len(list(Pcounts[i]))!=0:
        means.append(np.array(list(Pcounts[i].values())).mean())
        stds.append(np.array(list(Pcounts[i].values())).std())

In [ ]:
np.array(means).mean(),np.array(means).std()

In [ ]:
np.array(stds).mean(),np.array(stds).std()

In [ ]:
passage_lengths=[]
for i in range(len(df)):
    docid=str(df.iloc[i][0])
    relp = str(df.iloc[i][4])
    passage=data[docid][relp].translate(table).lower()
    passage=re.sub(r'\n', ' ', passage)
    passage=remove_stopwords(passage)
    words = word_tokenize(passage)
    passage_lengths.append(len(words))

In [ ]:
tf = np.array(list(Pcounts[0].values()))/passage_lengths[0]

In [ ]:
term_freqs=[]
for i in range(len(Pcounts)):
    term_freqs.append(np.array(list(Pcounts[i].values()))/passage_lengths[i])
    

In [ ]:
# Analyzing for the overlapping words, the term frequencies in the passages on average. 
means_tf=[]
stds_tf=[]
for i in range(len(term_freqs)):
    if len(list(term_freqs[i]))!=0:
        means_tf.append(term_freqs[i].mean())
        stds_tf.append(term_freqs[i].std())

In [ ]:
np.array(means_tf).mean(),np.array(means_tf).std()


In [ ]:
np.array(stds_tf).mean(),np.array(stds_tf).std()

In [ ]:
allpassages=[]
for i in data.keys():
    for j in data[i].keys():
        allpassages.append(data[i][j])

In [ ]:
counts = dict()      
for dicts in Pcounts:
    for word in list(dicts.keys()):        
        count=0
        for passages in allpassages:
            if word in passages:
                count+=1
                continue
        counts[word]=count        

In [ ]:
counts["evangelicalism"]

In [ ]:
import pickle
doc_freq_path = os.path.join(DATA_DIR, "processed/DocumentFreq.pickle")
with open(doc_freq_path, 'wb') as handle:
    pickle.dump(counts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
evangelicalismidf=np.log(50612/(counts["evangelicalism"]+1))

In [ ]:
tf[0]*evangelicalismidf

In [ ]:
term_freqs[0] 

In [ ]:
termfreq_idf=[]
for i,j in zip(term_freqs,Pcounts):
    templ=[]
    for k,l in zip(i,j.keys()):
        templ.append(k*np.log(50612/(counts[l]+1)))
        
    termfreq_idf.append(np.array(templ))    

In [ ]:
termfreq_idf[0]

In [ ]:
# Analyzing for the overlapping words how much they occur with respect to term freq_idf in the passages on average. 
means_tf_idf=[]
stds_tf_idf=[]
for i in range(len(termfreq_idf)):
    if len(list(termfreq_idf[i]))!=0:
        means_tf_idf.append(termfreq_idf[i].mean())
        stds_tf_idf.append(termfreq_idf[i].std())

In [ ]:
np.array(means_tf_idf).mean(),np.array(means_tf_idf).std()

In [ ]:
np.array(stds_tf_idf).mean(),np.array(stds_tf_idf).std()